In [ ]:
!pip install uv
!uv pip install  -r requirements.txt 

In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation and analysis
import numpy as np
import pandas as pd
from IPython.display import display

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

# Geospatial raster data handling with CRS support
import rioxarray as rxr

# Raster operations and spatial windowing
import rasterio
from rasterio.windows import Window

# Feature preprocessing and data splitting
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os 
from xgboost import XGBRegressor

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np

## Load Rivers and Dams data

In [ ]:
import geopandas as gpd
rivers = gpd.read_file("Rivers_Data/Rivers.shp")
rivers.head()

In [ ]:
rivers.crs

## Load the data we want to join

In [ ]:
water_training_df = pd.read_csv("water_quality_training_dataset.csv")
water_training_df.head()

In [ ]:
water_validation_df = pd.read_csv("submission_template.csv")
water_validation_df

In [ ]:
water_training_df.columns

In [ ]:
# define longtitude and latitude column names
lon_col = "Longitude"
lat_col = "Latitude"

water_training_gdf = gpd.GeoDataFrame(
    water_training_df,
    geometry=gpd.points_from_xy(water_training_df[lon_col], water_training_df[lat_col]),
    crs="EPSG:4326"  # Longtitude and latitude
)

print(water_training_gdf.crs)
water_training_gdf.head()

In [ ]:
# define longtitude and latitude column names
lon_col = "Longitude"
lat_col = "Latitude"

water_validation_gdf = gpd.GeoDataFrame(
    water_validation_df,
    geometry=gpd.points_from_xy(water_validation_df[lon_col], water_validation_df[lat_col]),
    crs="EPSG:4326"  # Longtitude and latitude
)

print(water_validation_gdf.crs)
water_validation_gdf.head()


In [ ]:
# Check for the training set
print(water_training_gdf[[lon_col, lat_col]].describe())
print(water_training_gdf[[lon_col, lat_col]].isna().sum())

In [ ]:
# Check for the validation set
print(water_validation_gdf[[lon_col, lat_col]].describe())
print(water_validation_gdf[[lon_col, lat_col]].isna().sum())


In [ ]:
# Project to the meter coordinate system
water_m = water_training_gdf.to_crs("EPSG:3857")
river_m = rivers.to_crs("EPSG:3857")
water_v_m = water_validation_gdf.to_crs("EPSG:3857")

In [ ]:
river_m.columns

In [ ]:
# Join the points with the nearest river
combined_df = gpd.sjoin_nearest(
    water_m,
    river_m,
    how="left",
    distance_col="dist_to_river_m"
)

combined_df.head()

In [ ]:
# Join the points with the nearest river
combined_validation_df = gpd.sjoin_nearest(
    water_v_m,
    river_m,
    how="left",
    distance_col="dist_to_river_m"
)

combined_validation_df.head()

In [ ]:
combined_df["dist_to_river_m"].describe(percentiles=[0.5,0.9,0.95,0.99])

In [ ]:
combined_validation_df["dist_to_river_m"].describe(percentiles=[0.5,0.9,0.95,0.99])

In [ ]:
# Set the limit to 500m to avoid illogical join. 
max_dist = 500  # 500m
combined_df.loc[combined_df["dist_to_river_m"] > max_dist] = None
combined_validation_df.loc[combined_df["dist_to_river_m"] > max_dist] = None

In [ ]:
water_river = combined_df.drop(columns="geometry") 
water_river_validation = combined_validation_df.drop(columns="geometry") 

water_river.to_csv("water_with_river_training.csv", index=False)
water_river_validation.to_csv("water_with_river_validation.csv", index=False)


In [ ]:
session.sql("""
    PUT file://water_with_river_training.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge-Dev"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")



In [ ]:
session.sql("""
    PUT file://water_with_river_validation.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge-Dev"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")



In [ ]:
water_river.head()

## Function for joining the data

In [ ]:
import pandas as pd
import geopandas as gpd

def to_points_gdf(
    df: pd.DataFrame,
    lon_col: str = "Longitude",
    lat_col: str = "Latitude",
    crs: str = "EPSG:4326",
    drop_invalid: bool = True
) -> gpd.GeoDataFrame:
    """
    Convert a DataFrame with lon/lat columns into a GeoDataFrame of Point geometry.
    Optionally drops rows with invalid/missing coordinates.
    """
    df = df.copy()

    # basic checks
    if lon_col not in df.columns or lat_col not in df.columns:
        raise ValueError(f"Missing lon/lat columns: '{lon_col}', '{lat_col}'")

    # numeric coercion
    df[lon_col] = pd.to_numeric(df[lon_col], errors="coerce")
    df[lat_col] = pd.to_numeric(df[lat_col], errors="coerce")

    # invalid coords
    invalid = (
        df[lon_col].isna() |
        df[lat_col].isna() |
        (df[lon_col] < -180) | (df[lon_col] > 180) |
        (df[lat_col] < -90)  | (df[lat_col] > 90)
    )

    if drop_invalid and invalid.any():
        df = df.loc[~invalid].copy()

    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs=crs
    )
    return gdf


In [ ]:
def join_nearest_layer(
    points_gdf: gpd.GeoDataFrame,
    layer_gdf: gpd.GeoDataFrame,
    *,
    layer_keep_cols: list[str] | None = None,
    dist_col: str = "dist_to_layer_m",
    max_dist_m: float | None = None,
    metric_crs: str = "EPSG:3857",
    how: str = "left"
) -> gpd.GeoDataFrame:
    """
    Spatially join points to the nearest feature in a layer (river lines, dam polygons, etc.)
    Returns points with selected attributes from the layer + distance in meters.

    - layer_keep_cols: columns to bring back from layer (excluding geometry; geometry auto handled)
    - max_dist_m: if provided, any match beyond this distance will have joined columns set to NA
    """
    # Decide which layer columns to keep
    if layer_keep_cols is None:
        # default: bring back all columns
        cols = [c for c in layer_gdf.columns if c != "geometry"]
    else:
        missing = [c for c in layer_keep_cols if c not in layer_gdf.columns]
        if missing:
            raise ValueError(f"layer_keep_cols not found in layer_gdf: {missing}")
        cols = layer_keep_cols

    # Project both to metric CRS for distance correctness
    pts_m = points_gdf.to_crs(metric_crs)
    lyr_m = layer_gdf.to_crs(metric_crs)

    # drop the "index_right" column to avoid the same name conflicts when joining.
    pts_m = pts_m.drop(columns=["index_right"], errors="ignore")

    # join
    matched = gpd.sjoin_nearest(
        pts_m,
        lyr_m[cols + ["geometry"]],
        how=how,
        distance_col=dist_col
    )

    # apply max distance filter only to joined columns (keep original point columns)
    if max_dist_m is not None:
        too_far = matched[dist_col] > max_dist_m

        # columns that came from layer (these are the ones we want to null out if too far)
        joined_cols = cols

        matched.loc[too_far, joined_cols] = pd.NA

    return matched

In [ ]:
import geopandas as gpd
import pandas as pd

rivers = gpd.read_file("Rivers_Data/Rivers.shp")
dams = gpd.read_file("Dams_Data/South_Africa_Dams.shp") 

test_df = pd.read_csv("submission_template.csv") # the data we want to join

In [ ]:
# Transform the csv file to gdf
test_gdf = to_points_gdf(test_df, lon_col="Longitude", lat_col="Latitude")

In [ ]:
# Define the columns we want to extract, default is set to be all columns
# river_cols = []
#dam_cols = []

test_with_river = join_nearest_layer(
    test_gdf,
    rivers,
    # layer_keep_cols=river_cols,
    dist_col="dist_to_river_m",
    max_dist_m=500,  # The max distance between the nearest river, for now we set to 500 meters.
    metric_crs="EPSG:3857"
)

test_with_dam = join_nearest_layer(
    test_with_river,
    dams,
    # layer_keep_cols=dam_cols,
    dist_col="dist_to_dam_m",
    max_dist_m=5000,         # distance for dam to the point could be larger?
    metric_crs="EPSG:3857"
)


In [ ]:
test_with_dam.head()